In [1]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Comparator computation

This computes the Euclidean distance for each establishment from a base establishment of a consistent type (Academies, Maintained schools, SEN). Each establishment is given a weight against every other establishment and the top 60 for each establishment creates the comparator set for that establishment. This is repeated for all establishments in the establishment types set and then finally across all establishments across all establishment types.

In [2]:
import time 
import numpy as np
import pandas as pd
import src.pipeline.comparator_sets as comparators
import glob 
import os

start_time = time.time()
# Create and clean directory
from pathlib import Path
Path("output/comparator-sets").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/comparator-sets/*")
for f in files:
    os.remove(f)

# Prepare Academy and School Data

Here we prepare the academy and maintained school data by filling in missing values in NumberOfPupils, % Free School Meals and, % Sen with the mean (at this time). 

In [6]:
academy_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/academies.parquet"))
ms_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/maintained_schools.parquet"))
all_schools = comparators.prepare_data(pd.read_parquet("output/pre-processing/all_schools.parquet"))

# Compute the pupil and building comparators

This creates the comparators sets across both academy and maintained schools

In [4]:
ms_comparators = comparators.compute_comparator_set(ms_data)
academy_comparators = comparators.compute_comparator_set(academy_data)
mixed_comparators = comparators.compute_comparator_set(all_schools)

In [7]:
ms_comparators.to_parquet("output/comparator-sets/ms_comparators.parquet")
academy_comparators.to_parquet("output/comparator-sets/academy_comparators.parquet")
academy_comparators.to_parquet("output/comparator-sets/mixed_comparators.parquet")
ms_data.to_parquet("output/comparator-sets/maintained_schools.parquet")
academy_data.to_parquet("output/comparator-sets/academies.parquet")
all_schools.to_parquet("output/comparator-sets/all_schools.parquet")

In [8]:
ms_comparators.head(10)

,OfstedRating (name),Percentage SEN,Percentage Free school meals,Number of pupils,Total Internal Floor Area,Age Average Score,GOR (name),Pupil,Building
URN,,,,,,,,,
100000,Outstanding,2.952030,18.1,271.0,3628.0,142.0,London,"[100000, 102285, 102892, 101544, 102379, 10249...","[100000, 101110, 100010, 100245, 100125, 10002..."
100005,Outstanding,1.470588,44.3,107.5,1858.0,55.792578,London,"[100005, 100767, 100315, 101105, 100554, 10149...","[100005, 101253, 102700, 100882, 100474, 10047..."
100006,Good,0.000000,68.4,49.0,1523.0,59.797111,London,"[100006, 100391, 134919, 133749, 100994, 13526...","[100006, 100007, 134635, 133749, 108892, 13175..."
100007,Good,47.368421,100.0,19.0,1858.0,55.792578,London,"[100007, 134597, 100391, 100006, 131584, 10099...",NaN
100008,Good,3.428571,55.1,350.0,2951.0,117.0,London,"[100008, 100408, 100341, 131740, 100403, 10041...","[100008, 131773, 102823, 100584, 102884, 10081..."
100009,Good,2.827763,43.4,389.0,3021.0,134.643496,London,"[100009, 102649, 100240, 100560, 102094, 10091...","[100009, 100996, 131595, 100896, 100907, 10080..."
100010,Good,3.550296,60.8,327.0,3415.0,142.0,London,"[100010, 131340, 131218, 101110, 101029, 13174...","[100010, 100245, 100783, 130358, 100000, 10042..."
100011,Good,1.015228,31.8,378.0,3817.0,72.965418,London,"[100011, 100229, 101815, 102399, 102635, 10190...","[100011, 102830, 101524, 136809, 102189, 10212..."
100013,Outstanding,3.000000,51.5,199.0,2552.0,7.0,London,"[100013, 100619, 100447, 100634, 100029, 10112...","[100013, 102921, 134307, 102568, 100732, 10186..."


### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')